<a href="https://colab.research.google.com/github/soheilpaper/twitter-personality-classification1/blob/master/Automate_robat/Groq_autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogen-agentchat~=0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.3/382.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.3 MB/s eta 0:00:00


In [2]:
import os
from autogen import AssistantAgent, UserProxyAgent

# Configure Groq API
llm_config = {
    "config_list": [
        {
            "model": "llama3-8b-8192",  # Example model supported by Groq
            "api_key": "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", #os.environ.get("GROQ_API_KEY"),
            "base_url": "https://api.groq.com/openai/v1"
        }
    ]
}

# Initialize agents
assistant = AssistantAgent("assistant", llm_config=llm_config)
user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

# Start the chat
user_proxy.initiate_chat(
    assistant,
    message="Tell me a joke about NVDA and TESLA stock prices."
)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


user_proxy (to assistant):

Tell me a joke about NVDA and TESLA stock prices.

--------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

In [4]:
openai_key = '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'# @param{type:'string'}
config_list = [
    {
        'model': 'llama3-8b-8192',
        'api_key': openai_key , #'<your OpenAI API key here>',
        #"max_retry_period": 50,
        #"retry_wait_time": 10,
        "base_url": "https://api.groq.com/openai/v1"
    }]

In [6]:

import subprocess
from typing import Union, Dict
import autogen
from IPython import get_ipython

class IPythonUserProxyAgent(autogen.UserProxyAgent):
    def __init__(self, name: str, **kwargs):
        super().__init__(name, **kwargs)
        self._ipython = get_ipython()

    def generate_init_message(self, *args, **kwargs) -> Union[str, Dict]:
        return (
            super().generate_init_message(*args, **kwargs)
            + """
If you suggest code, the code will be executed in IPython."""
        )

    def run_code(self, code, **kwargs):
        result = self._ipython.run_cell("%%capture --no-display cap\n" + code)
        log = self._ipython.ev("cap.stdout")
        log += self._ipython.ev("cap.stderr")

        if result.result is not None:
            log += str(result.result)

        exitcode = 0 if result.success else 1

        if result.error_before_exec is not None:
            log += f"\n{result.error_before_exec}"
            exitcode = 1

        if result.error_in_exec is not None:
            log += f"\n{result.error_in_exec}"
            exitcode = 1

            # Check if the error message indicates that a module is not found
            if "No module named" in str(result.error_in_exec):
                missing_module = str(result.error_in_exec).split("'")[1]
                try:
                    # Use pip install to install the missing module
                    subprocess.check_call(["pip", "install", missing_module])

                    # Retry running the code after installation
                    result_after_install = self._ipython.run_cell("%%capture --no-display cap\n" + code)

                    # Update log and exit code based on the result after installation
                    log = self._ipython.ev("cap.stdout") + self._ipython.ev("cap.stderr")
                    log += str(result_after_install.result) if result_after_install.result is not None else ""
                    exitcode = 0 if result_after_install.success else 1

                    if result_after_install.error_before_exec is not None:
                        log += f"\n{result_after_install.error_before_exec}"
                        exitcode = 1
                    if result_after_install.error_in_exec is not None:
                        log += f"\n{result_after_install.error_in_exec}"
                        exitcode = 1
                except Exception as install_error:
                    # Handle installation error
                    log += f"\nError installing {missing_module}: {install_error}"
                    exitcode = 1

        return exitcode, log, None

# Create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
   name="assistant",
   llm_config={
       "cache_seed": 42, # seed for caching and reproducibility
       "config_list": [ {'model': 'llama3-8b-8192', #'gpt-3.5-turbo',
        'api_key': openai_key , #'<your OpenAI API key here>',
         "base_url": "https://api.groq.com/openai/v1"
 }], # a list of OpenAI API configurations
       "temperature": 0, # temperature for sampling
   }, # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)

# Create a UserProxyAgent instance named "user_proxy"
ipy_user = IPythonUserProxyAgent(
   "ipython_user_proxy",
   human_input_mode="NEVER",
   max_consecutive_auto_reply=10,
   is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
   or x.get("content", "").rstrip().endswith('"TERMINATE".'),
   code_execution_config={
       "work_dir": "./coding",
       "use_docker": False, # set to True or image name like "python:3" to use docker
   },
)

'''
task = "Please summarize one docx report content by an upload file option and then create a 30-second voice for that summarized in Persian and English languages. Please consider I have the OpenAI API key and other modules be free if possible"
ipy_user.initiate_chat(
   assistant,
   message=f"{task}",
)

'''

'\ntask = "Please summarize one docx report content by an upload file option and then create a 30-second voice for that summarized in Persian and English languages. Please consider I have the OpenAI API key and other modules be free if possible"\nipy_user.initiate_chat(\n   assistant,\n   message=f"{task}",\n)\n\n'

In [7]:


task = "If possible creat one folder with name related to current task in current dir and run the code written for this and consider fill it's input vars or as the code be run  and ended and not as a loop as save the results  in docx word in current dir with name of iran if needs write second code to save results in docx file and give the saved docx file by running the second code too:(Create a single agent system to analyze the psychological impact of gender apartheid in Iran's Mahsa Amini movement and develop a waste management app as a prototype for open-source government. The agent should focus on the following tasks:  1. Analyze the psychological effects of gender apartheid on individuals and society in Iran, focusing on the impact of the Mahsa Amini movement. Provide insights on how to address these issues through new protocols and policies.  2. Design and implement a waste management app as a model for open-source government. Ensure the app can handle a $1 trillion market in waste management services. Include features for efficient resource allocation, real-time tracking, and community engagement.  3. Evaluate the ethical implications of using technology to address social and environmental issues in Iran. Provide recommendations for balancing individual rights with collective benefits.  4. Develop a comprehensive strategy for deploying the waste management app across Iran. Address potential challenges, required infrastructure, and necessary partnerships.  5. Analyze the long-term sustainability of the proposed waste management solution. Identify areas for improvement and suggest adaptations to ensure continued effectiveness over time.  The agent should work collaboratively within itself to produce a cohesive report addressing all aspects of the challenge. Use AutoGen's capabilities to generate detailed responses for each task, providing concrete examples and solutions where possible.  Note: This prompt has been modified to work within the constraints of AutoGen's current limitations while still capturing the essence of the original request. The multi-agent system approach has been replaced with a single agent that handles multiple tasks.  Citations: [1] https://www.reddit.com/r/AutoGenAI/comments/196wflt/autogen_error_occurred_while_processing_message/ [2] https://github.com/microsoft/autogen/issues/486 [3] https://github.com/microsoft/autogen/issues/1976 [4] https://microsoft.github.io/autogen/0.2/docs/FAQ/ [5] https://medium.com/@krtarunsingh/mastering-autogen-a-comprehensive-guide-to-next-generation-language-model-applications-b375d9b4dc6d [6] https://www.youtube.com/watch?v=YXWbatLW_mo [7] https://microsoft.github.io/autogen/0.2/docs/topics/llm_configuration/ [8] https://clintgoodman27.medium.com/a-practical-guide-for-using-autogen-in-software-applications-8799185d27ee [9] https://stackoverflow.com/questions/14841946/trouble-when-running-autogen-sh [10] https://microsoft.github.io/autogen/0.2/docs/notebooks/agentchat_function_call_code_writing/"# @param{type:'string'}
ipy_user.initiate_chat(
    assistant,
    message=f"{task}",
)

ipython_user_proxy (to assistant):

If possible creat one folder with name related to current task in current dir and run the code written for this and consider fill it's input vars or as the code be run  and ended and not as a loop as save the results  in docx word in current dir with name of iran if needs write second code to save results in docx file and give the saved docx file by running the second code too:(Create a single agent system to analyze the psychological impact of gender apartheid in Iran's Mahsa Amini movement and develop a waste management app as a prototype for open-source government. The agent should focus on the following tasks:  1. Analyze the psychological effects of gender apartheid on individuals and society in Iran, focusing on the impact of the Mahsa Amini movement. Provide insights on how to address these issues through new protocols and policies.  2. Design and implement a waste management app as a model for open-source government. Ensure the app can handle

BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}